In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
import sys
sys.path.append('/home/chenani/ownCloud/Workspaces/Eclipse/dataAnalysis/Sleep-current/src/')

In [2]:
%matplotlib 
#inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as pl
import numpy as np
import pickle as pkl
import os
import seaborn as sns
import pandas as pd
from signale.tools import locate,findNearest_vec
import matplotlib.gridspec as gridspec
from scipy.signal import gaussian 
import statsmodels.api as sm 

Using matplotlib backend: Qt4Agg


/home/chenani/anaconda/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
/home/chenani/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [3]:
aniPath = '/home/chenani/dataWork/ali/MECLesion_SleepData/'
anID = '3906'
dd = 'D1'
dday = '05-31-16_Day1'
spkPaths = [os.path.join(item[0],item[1])for item in locate('placeCells.spk',aniPath)]
trjPaths = [os.path.join(item[0],item[1])for item in locate('*.traj',aniPath)]
ltDB = pkl.load(open([os.path.join(item[0],item[1]) for item in locate('linearTrack'+dd+'.db','/home/chenani/ownCloud/') if item[0].find(anID)>0][0],'rb'))
Rtemp = pkl.load(open([os.path.join(item[0],item[1]) for item in locate('Rtemplate'+dd+'.nparr','/home/chenani/ownCloud/') if item[0].find(anID)>0][0],'rb'))
Ltemp = pkl.load(open([os.path.join(item[0],item[1]) for item in locate('Ltemplate'+dd+'.nparr','/home/chenani/ownCloud/') if item[0].find(anID)>0][0],'rb'))
RtempMod = np.delete(Rtemp,6)
lfp = pkl.load(open('/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3906/LinearTrack_05-31-16_Day1/02-begin1/CSC2.lfp'))
if hasattr(lfp,'signal_filtered'):
    lfp.filter(100,250)
if hasattr(lfp,'hilbertAbsSmooth'):
    lfp.hilbertTransform()
selSpk = [item for item in spkPaths if item.find(dday)>0][0]
selTrj = [item for item in trjPaths if item.find(dday)>0][0]
selSpk,selTrj

('/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3906/LinearTrack_05-31-16_Day1/parrentData/placeCells.spk',
 '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3906/LinearTrack_05-31-16_Day1/parrentData/linearTrack_Trajectory.traj')

In [4]:
spk = pkl.load(open(selSpk,'rb'))
trj = pkl.load(open(selTrj,'rb'))
spk.changeTimeUnit('s')
stList = spk.time_slice(trj.t_start,trj.t_stop)
#Interpolating trajectory
t = trj.times
It = np.arange(0,trj.times.size)
Itp = np.arange(0,trj.times.size,0.15)
tInterp = np.interp(Itp,It,t) #interpolated traj times
x = trj.places[:,0]
xp = np.interp(tInterp,t,x)       #interpolated traj Xposition
popsTi = [4544.7,4564.5,4567.9,4571.1,4577.6]
popsTf = [4545.1,4564.8,4568.3,4571.4,4578.0]

In [5]:
sns.set_context("paper",font_scale=1.6)
colors = ["windows blue", "amber", "greyish", "faded green", "dusty purple"]
leftRight = sns.color_palette(['#f9ba32','#426e86'])
seabC = sns.xkcd_palette(colors)
seabP = sns.color_palette("Paired")
flatui = sns.color_palette([  "#34495e","#e74c3c", "#2ecc71","#9b59b6", "#3498db", "#95a5a6"])
#sns.palplot(flatui)
sns.set_style('ticks')
kr = np.array(['k','r'])
###############BASIC LAYOUT
figure = pl.figure(figsize = [16,9])
gs1 = gridspec.GridSpec(9, 1)
gs1.update(left=0.05, right=0.45,top=0.99,bottom=0.65, wspace=0.01)
ax0 = pl.subplot(gs1[2:, :])
ax1 = pl.subplot(gs1[:2,:],sharex=ax0)
ax1tw = ax1.twinx()
ax1.set_xticks([])
gs2 = gridspec.GridSpec(9, 15)
gs2.update(left=0.1, right=0.45,top=0.60, hspace=0.05)
ax2 = pl.subplot(gs2[:2,:2])
ax7 = pl.subplot(gs2[2:4,:2],sharex=ax2)
ax3 = pl.subplot(gs2[:2,3:5])
ax8 = pl.subplot(gs2[2:4,3:5],sharex=ax3)
ax4 = pl.subplot(gs2[:2,6:8])
ax9 = pl.subplot(gs2[2:4,6:8],sharex=ax4)
ax5 = pl.subplot(gs2[:2,9:11])
ax10 = pl.subplot(gs2[2:4,9:11],sharex=ax5)
ax6 = pl.subplot(gs2[:2,12:14])
ax11 = pl.subplot(gs2[2:4,12:14],sharex=ax6)
##Lesion ax
axl2 = pl.subplot(gs2[5:7,:2])
axl7 = pl.subplot(gs2[7:,:2],sharex=axl2)
axl3 = pl.subplot(gs2[5:7,3:5])
axl8 = pl.subplot(gs2[7:,3:5],sharex=axl3)
axl4 = pl.subplot(gs2[5:7,6:8])
axl9 = pl.subplot(gs2[7:,6:8],sharex=axl4)
axl5 = pl.subplot(gs2[5:7,9:11])
axl10 = pl.subplot(gs2[7:,9:11],sharex=axl5)
axl6 = pl.subplot(gs2[5:7,12:14])
axl11 = pl.subplot(gs2[7:,12:14],sharex=axl6)
#gs1.tight_layout(figure,rect=[0, 0.35, 0.45,1])
#gs2.tight_layout(figure,rect=[0.05, 0, 0.45,0.3])
axarrAll = [ax0,ax1,ax1tw,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9,ax10,ax11]
for axx in axarrAll[4:]:
    pl.setp(axx.get_yticklabels(), visible=False)

In [7]:
lfp.filter(6,10)
lfp.hilbertTransform()
thtf = lfp.signal_filtered
thts = np.convolve(lfp.hilbertAbsolute,gaussian(1500,500),'same')
lfp.filter(150,200)
lfp.hilbertTransform()
Rts = np.convolve(lfp.hilbertAbsolute,gaussian(150,50),'same')

In [78]:
stList = spk.time_slice(4535,4595)
ax1.plot(lfp.timeAxis/1e3,thts,'c',lw=1.5,label=r'$\theta$',zorder=-1)
ax1tw.plot(lfp.timeAxis/1e3,Rts,'r',lw=1.0,label='Ripp',zorder=-1)
ax1.set_ylim(-3e6,5.4e6)
ax1tw.set_ylim(-3e3,3.2e5)
twax = ax0.twinx()
axarr = [ax0,twax,ax1,ax1tw]
for item in ltDB[ltDB[:,2]==1]:
    ax0.plot(tInterp[item[0]:item[1]],xp[item[0]:item[1]],c=leftRight[0],lw=5,zorder=1)#7f64b9
for item in ltDB[ltDB[:,2]==-1]:
    ax0.plot(tInterp[item[0]:item[1]],xp[item[0]:item[1]],c=leftRight[1],lw=5,zorder=1)#c36785
for pair in zip(popsTi,popsTf):
    ax0.axvspan(pair[0]-0.1,pair[1]+0.1,color=seabC[3],alpha=0.75,lw=0)#FFD700
for tIdx,cIdx in enumerate(RtempMod):
    zug  = stList[cIdx]
    twax.scatter(zug.spike_times,tIdx*np.ones(zug.spike_times.size),marker="|",c='k',lw=0.75)
'''for axx,pair in zip([ax2,ax3,ax4,ax5,ax6],zip(popsTi,popsTf)):
    stCut = stList.time_slice(pair[0]-0.001,pair[1]+0.001)
    for tIdx,cIdx in enumerate(RtempMod):
        zug  = stCut[cIdx]
        axx.scatter(zug.spike_times,tIdx*np.ones(zug.spike_times.size),marker="|",c='k',lw=2)
        axx.set_xticks([stCut.t_start,stCut.t_stop])
        axx.set_xticklabels([0,round(stCut.t_stop-stCut.t_start,1)])
   ''' 
ax0.set_ylabel('Position (m)')
#ax2.set_ylabel('Cells')
#ax2.set_xlabel('Time (s)')
#ax2.set_yticks([0,20,40])
#ax2.set_yticklabels([])
pl.setp(ax1.get_xticklabels(), visible=False)#ax1tw.set_ylim([0,800])
pl.setp(ax1tw.get_yticklabels(), visible=False)#
ax1tw.set_xticklabels([])
ax1tw.set_yticklabels([])
#ax1.xaxis.tick_bottom()
ax1.set_yticklabels([])
twax.set_ylim(-4,50)
ax0.set_xlabel('Time (s)')
twax.set_yticks([0,len(RtempMod)])
twax.set_yticklabels([len(RtempMod),0])
twax.set_xticks([4540,4555,4570,4585])
twax.set_xticklabels([0,15,30,45])
ax0.set_yticks([0.4,1.6])
twax.set_ylabel('Cell #',rotation=270)
ax0.set_xlim(4535,4595)
############LEGENDS
l1 = ax1.legend(fontsize=16)
l1.set_zorder(20)
l1.set_frame_on(False)
l1.set_alpha(0.9)
l1.set_bbox_to_anchor([1.2,1.0])
for ll in l1.legendHandles:
    ll.set_linewidth(2)
##----------------
l1w = ax1tw.legend(fontsize=16)
l1w.set_zorder(20)
l1w.set_frame_on(False)
l1w.set_alpha(0.9)
l1w.set_bbox_to_anchor([1.25,0.4])
for ll in l1w.legendHandles:
    ll.set_linewidth(2)
#figure.tight_layout()

In [13]:
for axx in axarr[:]:
    axx.cla()
    axx.set_yticks([])

In [20]:
for axx in axarrAll[3:]:
    axx.cla()
    axx.set_yticks([])

In [79]:
awro = pd.read_pickle('/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/control/Rat3906Control/dataFiles/awakeReplays-3906-D1-RO.db')
Rtemp = pd.read_pickle('/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/control/Rat3906Control/dataFiles/RtemplateD1.nparr')
Ltemp = pd.read_pickle('/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/control/Rat3906Control/dataFiles/LtemplateD1.nparr')
lapsDB = pd.read_pickle('/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/control/Rat3906Control/dataFiles/trajectory/linearTrackD1.db')
spk = pkl.load(open('/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3906/LinearTrack_05-31-16_Day1/parrentData/placeCells.spk'))
trj = pkl.load(open('/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3906/LinearTrack_05-31-16_Day1/parrentData/linearTrack_Trajectory.traj'))
sel_examples = [8,60,87,122,140]

In [80]:
axx = [ax2,ax3,ax4,ax5,ax6]
bxx = [ax7,ax8,ax9,ax10,ax11]
for ii,idx in enumerate(sel_examples):
    row = awro.loc[idx]
    ax = axx[ii]
    bx = bxx[ii]
    zug = spk.time_slice(row.ti-1e0,row.tf+1e0)
    zug.rasterPlot(temp=Ltemp,lw=3,clr=leftRight[0],meanClr='k',fig=figure,ax=ax,activeonly=True,yticks=False)# dgc[1]
    zug.rasterPlot(temp=Rtemp,lw=3,clr=leftRight[1],meanClr='k' ,fig=figure,ax=bx,activeonly=True,yticks=False)#'#ff0038'
    bx.set_xticks([row.ti,row.tf])
    bx.set_xticklabels([0,int(row.tf-row.ti)])
    #bx.set_xlabel('Time (ms)')
    bx.set_xlim(zug.t_start,zug.t_stop)
    axYmax = ax.get_ylim()
    bxYmax = bx.get_ylim()
    ax.set_ylim(axYmax[0],axYmax[1]+4)
    bx.set_ylim(bxYmax[0],bxYmax[1]+4)
    ax.text(round(zug.t_stop-4*(zug.t_stop-zug.t_start)/10),int(axYmax[1]+1.5),'r = '+str(round(row.Lcorr,2)),fontsize=12,ha='center', va='center')#, transform=ax.transAxes)
    bx.text(round(zug.t_stop-4*(zug.t_stop-zug.t_start)/10),int(bxYmax[1]+1.5),'r = '+str(round(row.Rcorr,2)),fontsize=12,ha='center', va='center')#, transform=bx.transAxes)
    ax.set_ylabel('')
    bx.set_ylabel('')
    ax.set_xticklabels(['',''])
    ax.set_yticklabels(['',''])
    bx.set_yticklabels(['',''])
    bx.set_xticklabels([0,int(zug.t_stop-zug.t_start)])
    ax.xaxis.tick_bottom()
    bx.xaxis.tick_bottom()
    ax.yaxis.tick_left()
    bx.yaxis.tick_left()
    if ii:
        ax.set_xlabel('')
        bx.set_xlabel('')
    pl.setp(ax.get_xticklabels(), visible=False)
    #bx.set_xlabel(int(zug.t_stop-zug.t_start),fontsize=16)

In [7]:
awro = pd.read_pickle('/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/lesion/Rat434Lesion/dataFiles/awakeReplays-434-D1-RO.db')
Rtemp = pd.read_pickle('/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/lesion/Rat434Lesion/dataFiles/RtemplateD1.nparr')
Ltemp = pd.read_pickle('/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/lesion/Rat434Lesion/dataFiles/LtemplateD1.nparr')
lapsDB = pd.read_pickle('/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/lesion/Rat434Lesion/dataFiles/linearTrackD1.db')
spk = pkl.load(open('/home/chenani/dataWork/ali/MECLesion_SleepData/Magda/Rat434Lesion/LinearTrack_0-0-0_Day1/parrentData/placeCells.spk'))
trj = pkl.load(open('/home/chenani/dataWork/ali/MECLesion_SleepData/Magda/Rat434Lesion/LinearTrack_0-0-0_Day1/parrentData/linearTrack_Trajectory.traj'))
sel_examples = [176,150,55,105,128]

In [11]:
laxx = [axl2,axl3,axl4,axl5,axl6]
lbxx = [axl7,axl8,axl9,axl10,axl11]
for ii,idx in enumerate(sel_examples):
    row = awro.loc[idx]
    print row.tSub,row.lSub,row.rSub
    ax = laxx[ii]
    bx = lbxx[ii]
    zug = spk.time_slice(row.ti-1e0,row.tf+1e0)
    zug.rasterPlot(temp=Ltemp,lw=3,clr=leftRight[0],meanClr='k',fig=figure,ax=ax,activeonly=True,yticks=False)# dgc[1]
    zug.rasterPlot(temp=Rtemp,lw=3,clr=leftRight[1],meanClr='k' ,fig=figure,ax=bx,activeonly=True,yticks=False)#'#ff0038'
    bx.set_xticks([row.ti,row.tf])
    bx.set_xticklabels([0,int(row.tf-row.ti)])
    bx.set_xlabel('Time (ms)')
    bx.set_xlim(zug.t_start,zug.t_stop)
    axYmax = ax.get_ylim()
    bxYmax = bx.get_ylim()
    ax.set_ylim(axYmax[0],axYmax[1]+4)
    bx.set_ylim(bxYmax[0],bxYmax[1]+4)
    ax.text(round(zug.t_stop-4*(zug.t_stop-zug.t_start)/10),int(axYmax[1]+1.5),'r = '+str(round(row.Lcorr,2)),fontsize=12,ha='center', va='center')#, transform=ax.transAxes)
    bx.text(round(zug.t_stop-4*(zug.t_stop-zug.t_start)/10),int(bxYmax[1]+1.5),'r = '+str(round(row.Rcorr,2)),fontsize=12,ha='center', va='center')#, transform=bx.transAxes)
    ax.set_ylabel('')
    bx.set_ylabel('')
    ax.set_xticklabels(['',''])
    ax.set_yticklabels(['',''])
    bx.set_yticklabels(['',''])
    bx.set_xticklabels([0,int(zug.t_stop-zug.t_start)])
    ax.xaxis.tick_bottom()
    bx.xaxis.tick_bottom()
    ax.yaxis.tick_left()
    bx.yaxis.tick_left()
    if ii:
        ax.set_xlabel('')
        bx.set_xlabel('')
    pl.setp(ax.get_xticklabels(), visible=False)
    #bx.set_xlabel(int(zug.t_stop-zug.t_start),fontsize=16)

17.0 14.0 11.0
30.0 25.0 21.0
26.0 20.0 17.0
25.0 21.0 19.0
21.0 18.0 15.0


In [86]:
#figure.savefig('/home/chenani/ownCloud/Plots/Lesion-Project/RO-Raster.pdf')

In [9]:
row

ti           2.303721e+07
tf           2.303752e+07
tp           2.303744e+07
tSub         2.100000e+01
lSub         1.800000e+01
rSub         1.500000e+01
Lcorr       -2.425181e-01
Rcorr        6.071429e-01
Lpv          1.000000e+00
Rpv          4.000000e-02
animal_ID    4.340000e+02
Name: 128, dtype: float64

In [83]:
gs2.update(left=0.05, right=0.48,top=0.55, hspace=0.02,wspace=0.001)

# Not plotted!!!

In [408]:
fireX = []
for idx in RtempMod:
    cell = spk[idx]
    validTimes = np.intersect1d(cell.right_spike_times[cell.right_spike_times/1e3>tInterp[0]],cell.right_spike_times[cell.right_spike_times/1e3<tInterp[-1]])
    cellXpos = xp[np.array([np.power((tInterp - validTimes[ijk]/1e3),2).argmin() for ijk in range(validTimes.size)])]
    fireX.append(cellXpos)
    #sns.kdeplot(cellXpos)

In [422]:
for item in fireX[20:30]:
    sns.kdeplot(item,bw=0.1)

In [411]:
for iii,item in enumerate(fireX):
    dens = sm.nonparametric.KDEUnivariate(item)
    dens.fit()
    xIn = np.linspace(0,2,100) 
    probs = dens.evaluate(xIn)
    pl.plot(xIn,iii+(probs/probs.max()))

In [406]:
for cId in spk.id_list[:]:
    cell = spk[cId]
    LspkTimes = np.array([])
    RspkTimes = np.array([])
    for lap in lapsDB:
        if lap[-1]<0:
            LspkTimes = np.append(LspkTimes,np.intersect1d(cell.spike_times[cell.spike_times/1e3>tInterp[lap[0]]],cell.spike_times[cell.spike_times/1e3<tInterp[lap[1]]]))
        else:
            RspkTimes = np.append(RspkTimes,np.intersect1d(cell.spike_times[cell.spike_times/1e3>tInterp[lap[0]]],cell.spike_times[cell.spike_times/1e3<tInterp[lap[1]]]))
    spk[cId].left_spike_times = LspkTimes
    spk[cId].right_spike_times = RspkTimes

In [407]:
cell.right_spike_times

array([ 3692007.  ,  3752177.25,  3786147.25, ...,  5433602.5 ,
        5433691.5 ,  5433820.  ])

In [8]:
lfp.hilbertTransform()

array([ 272.39753414,  295.80887551,  319.59835586, ...,  205.39155491,
        227.1267598 ,  249.47535331])

In [ ]:
ff,axx = pl.subplots(1,1,figsize=[10,20])
ti = stList.t_stop-200
counter = 1
while ti<stList.t_stop:
    spkCut = stList.time_slice(ti,ti+10)
    axx.set_xticks([])
    axx.set_xticklabels([])
    spikkes = np.zeros([30,RtempMod.size])
    spkBins = np.linspace(0,5,31)
    for tIdx,cIdx in enumerate(RtempMod):
        zug  = spkCut[cIdx]
        xspkT = zug.spike_times-spkCut.t_start
        hst,bns = np.histogram(xspkT,bins=spkBins)
        spikkes[hst>0,tIdx]+=1
    for iii in range(30):
        axx.pcolor(spikkes[:iii,:].T)#,marker="|",c='k',lw=2)
        #axx.set_ylim(-4,50)
        axx.set_xlim([0,30])
        ff.savefig('/home/chenani/ownCloud/Plots/Lesion-Project/anime/raster_'+str(counter)+'.jpg')
        counter+=1
    axx.cla()
    ti+=10
    print ti


In [ ]:
ff,axx = pl.subplots(1,1,figsize=[10,20])
ti = stList.t_stop-300
dividesecondTo = 10
spkCut = stList.time_slice(ti,stList.t_stop)
axx.set_xticks([])
axx.set_xticklabels([])
duration = int(np.diff(spkCut.time_parameters())[0])
spikkes = np.zeros([dividesecondTo*duration,RtempMod.size])
spkBins = np.linspace(0,duration,dividesecondTo*duration+1)
for tIdx,cIdx in enumerate(RtempMod):
    zug  = spkCut[cIdx]
    xspkT = zug.spike_times-spkCut.t_start
    hst,bns = np.histogram(xspkT,bins=spkBins)
    spikkes[hst>0,tIdx]+=1
for iii in range(spikkes.shape[0]):
    mtx = spikkes[iii:iii+10*dividesecondTo,:]
    axx.pcolor(mtx.T)
    axx.set_xlim([0,10*dividesecondTo])
    axx.set_xlabel('Time',fontsize=34)
    axx.set_xticks([])
    axx.set_ylabel('Cells',fontsize=34)
    ff.savefig('/home/chenani/ownCloud/Plots/Lesion-Project/anime/raster_'+str(iii)+'.jpg')
    axx.cla()
